In [27]:
import torch
import triton
import triton.language as tl
import time

In [28]:
def is_cuda():
    return triton.runtime.driver.active.get_current_target().backend == "cuda"

In [29]:
def is_hip_mi200():
    target = triton.runtime.driver.active.get_current_target()
    return target.backend == 'hip' and target.arch == 'gfx90a'

In [30]:
"""
PA2 Part 2: MatMul+Relu+Add Fused Optimization.
The kernel uses several optimization techniques:

  1. Shared memory tiling.
  2. Register tiling.
  3. Cooperative fetching.
  4. Operator Fusion
  5. Write cache / epilogue fusion.

Fill in the missing parts (marked with TODO).
"""

# -----------------------------------------------------------------------------
# Tiling parameters - You will need to change these to achieve better results.
# -----------------------------------------------------------------------------
BLOCK_M = 128  # Tile size in the M dimension.
BLOCK_N = 256 # Tile size in the N dimension.
BLOCK_K = 32 # Tile size in the K dimension.

# -----------------------------------------------------------------------------
# Triton Kernel: Matrix Multiplication + ReLU + Add
#
# The kernel uses:
#   Step 1: Tile assignment (each kernel computes a tile of C)
#   Step 2: Shared memory tiling + Cooperative Fetching: Load tiles of A and B.
#   Step 3: Register tiling: Use a register accumulator.
#   Step 4: Add and ReLU fusion
#   Step 5: Write cache/Epilogue: Write the final tile back to global memory.
# -----------------------------------------------------------------------------
@triton.jit
def matmul_add_relu_kernel_fp16(
    a_ptr, b_ptr, c_ptr, d_ptr,
    M: tl.constexpr, N: tl.constexpr, K: tl.constexpr,
    stride_am: tl.constexpr, stride_ak: tl.constexpr,
    stride_bk: tl.constexpr, stride_bn: tl.constexpr,
    stride_cm: tl.constexpr, stride_cn: tl.constexpr,
    stride_dm: tl.constexpr, stride_dn: tl.constexpr,
    BLOCK_M: tl.constexpr, BLOCK_N: tl.constexpr, BLOCK_K: tl.constexpr,
):
    # -------------------------------------------------------------------------
    # Step 1: Tile: Assignment
    #
    # Each kernel instance is mapped to a tile in the output matrix C.
    # Compute the starting indices (m_start, n_start) for this tile.
    # -------------------------------------------------------------------------
    # TODO: Compute the tile indices using program_id(0) for M and program_id(1) for N.
    idx_i = tl.program_id(0) * BLOCK_M + tl.arange(0, BLOCK_M)
    idx_j = tl.program_id(1) * BLOCK_N + tl.arange(0, BLOCK_N)
    X_C = tl.broadcast_to(idx_i[:, None], (BLOCK_M, BLOCK_N))
    Y_C = tl.broadcast_to(idx_j[None, :], (BLOCK_M, BLOCK_N))
    mask_C = (X_C < M) & (Y_C < N)
    idx_C = X_C * N + Y_C
    shared_C = tl.load(c_ptr + idx_C, mask=mask_C)

    # -------------------------------------------------------------------------
    # Step 2: Register Tiling
    # -------------------------------------------------------------------------
    # TODO: Initialize the accumulator "acc" with zeros (dtype: float16).
    acc = tl.zeros([BLOCK_M, BLOCK_N], dtype=tl.float16)

    # -------------------------------------------------------------------------
    # Step 3: Shared Memory Tiling & Cooperative Fetching.
    # Compute pointers to the sub-tiles of A and B that are needed to compute
    # the current C tile. The offsets here serve to load BLOCK_SIZE_M x BLOCK_SIZE_K
    # and BLOCK_SIZE_K x BLOCK_SIZE_N blocks from A and B respectively.
    # -------------------------------------------------------------------------
    # TODO: Finish code below
    for k in range(0, K, BLOCK_K):
        idx_k = k + tl.arange(0, BLOCK_K)

        X_A = tl.broadcast_to(idx_i[:, None], (BLOCK_M, BLOCK_K))
        Y_A = tl.broadcast_to(idx_k[None, :], (BLOCK_M, BLOCK_K))
        mask_A = (X_A < M) & (Y_A < K)
        idx_A = X_A * K + Y_A
        shared_A = tl.load(a_ptr + idx_A, mask=mask_A)

        X_B = tl.broadcast_to(idx_k[:, None], (BLOCK_K, BLOCK_N))
        Y_B = tl.broadcast_to(idx_j[None, :], (BLOCK_K, BLOCK_N))
        mask_B = (X_B < K) & (Y_B < N)
        idx_B = X_B * N + Y_B
        shared_B = tl.load(b_ptr + idx_B, mask=mask_B)

        acc += tl.dot(shared_A, shared_B, out_dtype=tl.float16)

    # -------------------------------------------------------------------------
    # Step 4: Apply ReLU and Add C to the accumulator
    # -------------------------------------------------------------------------
    acc = tl.maximum(acc + shared_C, 0)

    # -------------------------------------------------------------------------
    # Step 5: Write Cache / Epilogue Fusion: Write the computed tile to D.
    # -------------------------------------------------------------------------
    tl.store(d_ptr + idx_C, acc, mask=mask_C)

In [31]:
def matmul_add_relu_fp16(a: torch.Tensor, b: torch.Tensor, c: torch.Tensor) -> torch.Tensor:
    """
    Computes Output = ReLU(A @ B + C) using fp16 precision for maximum throughput.
    """
    M, K = a.shape
    K2, N = b.shape
    assert K == K2, "Incompatible dimensions"

    d = torch.empty((M, N), device=a.device, dtype=torch.float16)
    # Create launch grid
    grid = (triton.cdiv(M, BLOCK_M), triton.cdiv(N, BLOCK_N))

    matmul_add_relu_kernel_fp16[grid](
        a, b, c, d,
        M, N, K,
        a.stride(0), a.stride(1),
        b.stride(0), b.stride(1),
        c.stride(0), c.stride(1),
        d.stride(0), d.stride(1),
        BLOCK_M=BLOCK_M, BLOCK_N=BLOCK_N, BLOCK_K=BLOCK_K
    )
    return d

In [32]:
# Reference implementation using PyTorch
def reference_matmul_add_relu(A, B, C):
    result = torch.matmul(A, B).add(C).relu_()
    return result

In [33]:
# -----------------------------------------------------------------------------
# Accuracy Tests
# -----------------------------------------------------------------------------
if __name__ == "__main__":
    torch.manual_seed(0)
    a = torch.randn((512, 512), device=torch.device("cuda"), dtype=torch.float16)
    b = torch.randn((512, 512), device=torch.device("cuda"), dtype=torch.float16)
    c = torch.randn((512, 512), device=torch.device("cuda"), dtype=torch.float16)
    triton_output = matmul_add_relu_fp16(a, b, c)
    torch_output = reference_matmul_add_relu(a, b, c)
    print(f"triton_output_with_fp16_inputs={triton_output}")
    print(f"torch_output_with_fp16_inputs={torch_output}")
    rtol = 1e-2 if is_hip_mi200() else 0.032
    if torch.allclose(triton_output, torch_output, atol=0.15, rtol=rtol):
        print("✅ Triton and Torch match")
    else:
        diff = triton_output - torch_output
        abs_diff = torch.abs(diff)
        max_abs_diff = torch.max(abs_diff)
        print(f"❌ Triton and Torch differ: {max_abs_diff=}")

triton_output_with_fp16_inputs=tensor([[ 0.0000,  6.1250,  0.0000,  ..., 10.0625,  0.0000,  0.0000],
        [ 7.9102, 15.6328, 26.6094,  ..., 11.4609,  5.3750, 18.6250],
        [ 2.7246,  0.0000,  0.0000,  ...,  0.0000, 26.0781,  0.0000],
        ...,
        [ 0.4448, 75.1875,  0.0000,  ..., 26.2812,  0.0000,  0.0000],
        [ 6.9492,  1.1230,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [27.6094, 26.9531, 22.9219,  ..., 13.5391,  6.0508, 21.6250]],
       device='cuda:0', dtype=torch.float16)
torch_output_with_fp16_inputs=tensor([[ 0.0000,  6.1289,  0.0000,  ..., 10.0391,  0.0000,  0.0000],
        [ 7.9102, 15.6328, 26.6250,  ..., 11.4531,  5.3945, 18.6562],
        [ 2.7266,  0.0000,  0.0000,  ...,  0.0000, 26.1250,  0.0000],
        ...,
        [ 0.4316, 75.2500,  0.0000,  ..., 26.2812,  0.0000,  0.0000],
        [ 6.9570,  1.1260,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [27.6406, 26.9531, 22.9375,  ..., 13.5625,  6.0391, 21.6406]],
       device='cuda:0', dt

In [35]:
# -----------------------------------------------------------------------------
# Performance Benchmark
# IMPORTANT: DO NOT CHANGE THIS CODE.
# THIS IS THE EXACT CODE THAT WILL BE USED TO GRADE YOUR IMPLEMENTATION.
# ANY CHANGES TO THIS CODE (INCLUDING DIMENSIONS, REPEATS, etc.)
# WILL CAUSE YOU TO HAVE DIFFERENT SPEEDUP RESULTS.
# -----------------------------------------------------------------------------
M = 2048
K = 2048
N = 2048

# KEEP THESE MATRICES IN FP16. FP32 WILL NOT PROVIDE ACCURATE RESULTS
A = torch.randn((M, K), device="cuda", dtype=torch.float16)
B = torch.randn((K, N), device="cuda", dtype=torch.float16)
C = torch.randn((M, N), device="cuda", dtype=torch.float16)

# warmup
_ = matmul_add_relu_fp16(A, B, C)
_ = reference_matmul_add_relu(A, B, C)

REPEATS = 5000

# time your implementation
print("Triton implementation")
torch.cuda.synchronize()
start = time.perf_counter()
for _ in range(REPEATS):
    _ = matmul_add_relu_fp16(A, B, C)
torch.cuda.synchronize()
triton_time = (time.perf_counter() - start) / REPEATS

# time pytorch
print("PyTorch implementation")
torch.cuda.synchronize()
start = time.perf_counter()
for _ in range(REPEATS):
    _ = reference_matmul_add_relu(A, B, C)
torch.cuda.synchronize()
torch_time = (time.perf_counter() - start) / REPEATS

print(f"Performance comparison for matrix multiplication ({M}x{K} @ {K}x{N}):")
print(f"Triton implementation: {triton_time*1000:.2f} ms")
print(f"PyTorch implementation: {torch_time*1000:.2f} ms")

print(f"\nSpeedup of Triton vs PyTorch: {torch_time/triton_time:.2f}x")

Triton implementation
PyTorch implementation
Performance comparison for matrix multiplication (2048x2048 @ 2048x2048):
Triton implementation: 0.92 ms
PyTorch implementation: 1.19 ms

Speedup of Triton vs PyTorch: 1.30x


In [ ]:
# Write your grid search here.
BLOCK_M_test = [32, 64, 128]
BLOCK_N_test = [32, 64, 128, 256]
BLOCK_K_test = [32, 64]

A = torch.randn((2048, 2048), device="cuda", dtype=torch.float16)
B = torch.randn((2048, 2048), device="cuda", dtype=torch.float16)
C = torch.randn((2048, 2048), device="cuda", dtype=torch.float16)
REPEATS = 5000

for m in BLOCK_M_test:
    for n in BLOCK_N_test:
        for k in BLOCK_K_test:
            BLOCK_M = m
            BLOCK_N = n
            BLOCK_K = k

            print(f"BLOCK_M={m}, BLOCK_N={n}, BLOCK_K={k}\n")
            torch.cuda.synchronize()
            start = time.perf_counter()
            for _ in range(REPEATS):
                _ = matmul_add_relu_fp16(A, B, C)
            torch.cuda.synchronize()
            triton_time = (time.perf_counter() - start) / REPEATS
            print(f"time: {triton_time*1000:.2f} ms\n")

BLOCK_M=32, BLOCK_N=32, BLOCK_K=32

time: 5.30 ms

BLOCK_M=32, BLOCK_N=32, BLOCK_K=64

time: 4.63 ms

BLOCK_M=32, BLOCK_N=64, BLOCK_K=32

time: 3.18 ms

BLOCK_M=32, BLOCK_N=64, BLOCK_K=64

time: 2.69 ms

BLOCK_M=32, BLOCK_N=128, BLOCK_K=32

time: 2.06 ms

BLOCK_M=32, BLOCK_N=128, BLOCK_K=64

time: 1.55 ms

BLOCK_M=32, BLOCK_N=256, BLOCK_K=32

time: 1.48 ms

BLOCK_M=32, BLOCK_N=256, BLOCK_K=64

time: 1.83 ms

BLOCK_M=64, BLOCK_N=32, BLOCK_K=32

time: 4.39 ms

BLOCK_M=64, BLOCK_N=32, BLOCK_K=64

time: 3.43 ms

BLOCK_M=64, BLOCK_N=64, BLOCK_K=32

time: 2.34 ms

BLOCK_M=64, BLOCK_N=64, BLOCK_K=64

time: 2.10 ms

BLOCK_M=64, BLOCK_N=128, BLOCK_K=32

time: 1.69 ms

BLOCK_M=64, BLOCK_N=128, BLOCK_K=64

time: 1.48 ms

BLOCK_M=64, BLOCK_N=256, BLOCK_K=32

time: 1.21 ms

BLOCK_M=64, BLOCK_N=256, BLOCK_K=64

time: 1.27 ms

BLOCK_M=128, BLOCK_N=32, BLOCK_K=32

time: 3.36 ms

BLOCK_M=128, BLOCK_N=32, BLOCK_K=64

time: 2.41 ms

BLOCK_M=128, BLOCK_N=64, BLOCK_K=32

time: 1.72 ms

BLOCK_M=128, BLOCK_N